# Desafio Everis Spark

### Imports

In [194]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, current_date, year, lpad, udf

### Configurando Sessão

In [195]:
spark = SparkSession.builder \
   .master("local") \
   .appName("Desafio PySpark Everis") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()

### Criando o Schema do database

In [196]:
struct = StructType([ \
    StructField("codigo_cliente", IntegerType(), True), \
    StructField("nome", StringType(), True), \
    StructField("municipio", StringType(), True), \
    StructField("estado", StringType(), True), \
    StructField("data_nascimento", DateType(), True)])

### Carregando os dados do database

In [197]:
df = spark.read.csv("base.csv", schema=struct, sep=";", dateFormat="dd-MM-yyyy")
df.show()

+--------------+--------+-----------+-----------------+---------------+
|codigo_cliente|    nome|  municipio|           estado|data_nascimento|
+--------------+--------+-----------+-----------------+---------------+
|             1|    José|   Anápolis|        São Paulo|     1900-09-01|
|             2|    Igor|   Anápolis|        São Paulo|     1977-09-11|
|             3|Leonardo|   Anápolis|        São Paulo|     2000-12-21|
|             4|Humberto|Pato Branco|Rio Grande do Sul|     1964-11-13|
|             5|  Isaias|Pato Branco|Rio Grande do Sul|     2002-07-07|
|             6|   Lucas|       Taua|            Ceará|     1984-09-05|
+--------------+--------+-----------+-----------------+---------------+



## Casos para resolução

### Caso 1 – Adicionar 1 coluna com um contador sequencial por Município e ordenar por Estado.

In [198]:
windowSpec = Window.partitionBy("municipio").orderBy("estado")

In [199]:
df1 = df.withColumn("contador_municipio",row_number().over(windowSpec)).orderBy("estado")
df1.show()

+--------------+--------+-----------+-----------------+---------------+------------------+
|codigo_cliente|    nome|  municipio|           estado|data_nascimento|contador_municipio|
+--------------+--------+-----------+-----------------+---------------+------------------+
|             6|   Lucas|       Taua|            Ceará|     1984-09-05|                 1|
|             5|  Isaias|Pato Branco|Rio Grande do Sul|     2002-07-07|                 2|
|             4|Humberto|Pato Branco|Rio Grande do Sul|     1964-11-13|                 1|
|             1|    José|   Anápolis|        São Paulo|     1900-09-01|                 1|
|             2|    Igor|   Anápolis|        São Paulo|     1977-09-11|                 2|
|             3|Leonardo|   Anápolis|        São Paulo|     2000-12-21|                 3|
+--------------+--------+-----------+-----------------+---------------+------------------+



### Caso 2 - Adicionar 1 coluna com a Idade em anos e na coluna cod_cliente formatar o campo com 3 posições a esquerda completando com “0”.

In [200]:
df2 = df1.withColumn("idade", (year(current_date()) - year("data_nascimento"))) \
         .withColumn("codigo_cliente", lpad(col("codigo_cliente"), 3, '0').alias('s').cast('int'))
df2.show()

+--------------+--------+-----------+-----------------+---------------+------------------+-----+
|codigo_cliente|    nome|  municipio|           estado|data_nascimento|contador_municipio|idade|
+--------------+--------+-----------+-----------------+---------------+------------------+-----+
|             6|   Lucas|       Taua|            Ceará|     1984-09-05|                 1|   37|
|             4|Humberto|Pato Branco|Rio Grande do Sul|     1964-11-13|                 1|   57|
|             5|  Isaias|Pato Branco|Rio Grande do Sul|     2002-07-07|                 2|   19|
|             1|    José|   Anápolis|        São Paulo|     1900-09-01|                 1|  121|
|             3|Leonardo|   Anápolis|        São Paulo|     2000-12-21|                 3|   21|
|             2|    Igor|   Anápolis|        São Paulo|     1977-09-11|                 2|   44|
+--------------+--------+-----------+-----------------+---------------+------------------+-----+



In [201]:
df2.printSchema()

root
 |-- codigo_cliente: integer (nullable = true)
 |-- nome: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_nascimento: date (nullable = true)
 |-- contador_municipio: integer (nullable = true)
 |-- idade: integer (nullable = true)



### Caso 3 - Adicionar 1 coluna com a data de atualização, preenchendo com a data do dia da execução e retirar os caracteres especiais do campo Estado.

In [220]:
!pip install unidecode

     |████████████████████████████████| 241 kB 2.0 MB/s eta 0:00:01


In [221]:
from unidecode import unidecode

In [222]:
my_udf = udf(lambda x: unidecode(x), StringType())

In [223]:
df3 = df2.withColumn("data_atualizacao", current_date()) \
         .withColumn("estado", my_udf("estado"))
df3.show()

+--------------+--------+-----------+-----------------+---------------+------------------+-----+----------------+
|codigo_cliente|    nome|  municipio|           estado|data_nascimento|contador_municipio|idade|data_atualizacao|
+--------------+--------+-----------+-----------------+---------------+------------------+-----+----------------+
|             6|   Lucas|       Taua|            Ceara|     1984-09-05|                 1|   37|      2021-04-27|
|             4|Humberto|Pato Branco|Rio Grande do Sul|     1964-11-13|                 1|   57|      2021-04-27|
|             5|  Isaias|Pato Branco|Rio Grande do Sul|     2002-07-07|                 2|   19|      2021-04-27|
|             1|    José|   Anápolis|        Sao Paulo|     1900-09-01|                 1|  121|      2021-04-27|
|             2|    Igor|   Anápolis|        Sao Paulo|     1977-09-11|                 2|   44|      2021-04-27|
|             3|Leonardo|   Anápolis|        Sao Paulo|     2000-12-21|                 